In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!pip install pytorch-minimize &> /dev/null
!pip install jupyterplot &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

(IR_FLASH_fit)=
# Optimized IR FLASH for T1 fit

In [ ]:
#@title generate 5 IR FLASH images with increasing TI
# %% S0. SETUP env
import MRzeroCore as mr0
import pypulseq as pp
import numpy as np
import torch
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

experiment_id = 'FLASH_2D_Fit'

# %% S1. SETUP sys

# choose the scanner limits
system = pp.Opts(max_grad=28,grad_unit='mT/m',max_slew=150,slew_unit='T/m/s',
                 rf_ringdown_time=20e-6,rf_dead_time=100e-6,adc_dead_time=20e-6,grad_raster_time=50*10e-6)

TI = [0.1,0.5,1.0,5.0]
# Define FOV and resolution
fov = 200e-3
slice_thickness = 8e-3
Nread = 24    # frequency encoding steps/samples
Nphase = 24   # phase encoding steps/samples

##linear reordering
phenc = np.arange(-Nphase // 2, Nphase // 2, 1) / fov
# permvec = np.arange(0, Nphase, 1)
## centric reordering
permvec = sorted(np.arange(len(phenc)), key=lambda x: abs(len(phenc) // 2 - x))
## random reordering
#perm =np.arange(0, Nphase, 1);  permvec = np.random.permutation(perm)

# %% S2. DEFINE the sequence
def create_inv_rec(TI):
  seq = pp.Sequence()

  # Define rf events
  rf1, _, _ = pp.make_sinc_pulse(
      flip_angle=10 * np.pi / 180, duration=1e-3,
      slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
      system=system, return_gz=True
  )

  rf_inv = pp.make_block_pulse(flip_angle=180 * np.pi / 180, duration=1e-3, system=system)

  # Define other gradients and ADC events
  gx = pp.make_trapezoid(channel='x', flat_area=Nread / fov, flat_time=4e-3, system=system)
  adc = pp.make_adc(num_samples=Nread, duration=4e-3, phase_offset=0 * np.pi/180, delay=gx.rise_time, system=system)
  gx_pre = pp.make_trapezoid(channel='x', area=-gx.area / 2, duration=1e-3, system=system)
  gx_spoil = pp.make_trapezoid(channel='x', area=1.5 * gx.area, duration=2e-3, system=system)
  gy_spoil = pp.make_trapezoid(channel='y', area=1.5 * gx.area, duration=2e-3, system=system)

  rf_phase = 0
  rf_inc = 0
  rf_spoiling_inc = 117

  # ======
  # CONSTRUCT SEQUENCE
  # ======

  phenc_centr = phenc[permvec]
  for t in TI:
    seq.add_block(rf_inv)
    seq.add_block(pp.make_delay(t))
    seq.add_block(gx_spoil, gy_spoil)
    for ii in range(0, Nphase):  # e.g. -64:63

        rf1.phase_offset = rf_phase / 180 * np.pi   # set current rf phase

        adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC
        rf_inc = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]   # increase increment
        # increment additional pahse
        rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]

        seq.add_block(rf1)
        seq.add_block(pp.make_delay(0.005))
        gp = pp.make_trapezoid(channel='y', area=phenc_centr[ii], duration=1e-3, system=system)
        seq.add_block(gx_pre, gp)
        seq.add_block(adc, gx)
        gp = pp.make_trapezoid(channel='y', area=-phenc_centr[ii], duration=1e-3, system=system)
        seq.add_block(gx_spoil, gp)
    seq.add_block(pp.make_delay(10))
  return seq

seq = create_inv_rec(TI)

# %% S3. CHECK, PLOT and WRITE the sequence  as .seq
# Check whether the timing of the sequence is correct
ok, error_report = seq.check_timing()
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed. Error listing follows:')
    [print(e) for e in error_report]

# PLOT sequence
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, figid=(11,12))

# Prepare the sequence output for the scanner
seq.set_definition('FOV', [fov, fov, slice_thickness])
seq.set_definition('Name', 'gre')
seq.write(experiment_id + '.seq')


# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
sz = [Nread, Nread]
# (i) load a phantom object from file
phantom = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
phantom = phantom.interpolate(sz[0], sz[1], 1)
# Manipulate loaded data
phantom.T2dash[:] = 30e-3
phantom.D *= 0
phantom.B0 *= 1    # alter the B0 inhomogeneity
# Store PD and B0 for comparison
PD = phantom.PD
B0 = phantom.B0
T1 = phantom.T1
phantom.plot()
# Convert Phantom into simulation data
obj_p = phantom.build()


# %% S5:. SIMULATE  the external.seq file and add acquired signal to ADC plot
# Read in the sequence
seq0 = mr0.Sequence.import_file(experiment_id + '.seq')
seq0.plot_kspace_trajectory()
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p, print_progress=False)

# PLOT sequence with signal in the ADC subplot
plt.close(11);plt.close(12)
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, signal=signal.numpy())

idx = len(TI)-1

fig = plt.figure()  # fig.clf()
plt.subplot(411)
plt.title('ADC signal')
kspace = torch.reshape((signal), (len(TI),Nphase, Nread)).clone().permute(0,2,1)
plt.plot(torch.real(signal), label='real')
plt.plot(torch.imag(signal), label='imag')

ipermvec = np.argsort(permvec)

kspace=kspace[:,:,ipermvec]
# this adds ticks at the correct position szread
major_ticks = np.arange(0, len(TI)*Nphase * Nread, Nread)
ax = plt.gca()
ax.set_xticks(major_ticks)
ax.grid()

# fftshift
spectrum = torch.fft.fftshift(kspace,dim=(1,2))
# FFT
space = torch.fft.fft2(spectrum)
# fftshift
space = torch.fft.ifftshift(space,dim=(1,2))

plt.subplot(345)
plt.title('k-space')
plt.imshow(np.abs(kspace[idx].numpy()))
plt.subplot(349)
plt.title('log. k-space')
plt.imshow(np.log(np.abs(kspace[idx].numpy())))

plt.subplot(346)
plt.title('FFT-magnitude')
plt.imshow(np.abs(space[idx].numpy()))
plt.colorbar()
plt.subplot(3, 4, 10)
plt.title('FFT-phase')
plt.imshow(np.angle(space[idx].numpy()), vmin=-np.pi, vmax=np.pi)
plt.colorbar()

# % compare with original phantom obj_p.PD
plt.subplot(348)
plt.title('phantom PD')
plt.imshow(PD)
plt.subplot(3, 4, 12)
plt.title('phantom B0')
plt.imshow(B0)

#Fit

We now copy the true T1 map as T1_target. Initialize the T1 of our object with arbitrary values. Make the T1 of our object optimizable. And define our signal generated above as target.

In [ ]:
T1_target = phantom.T1 # Create Target T1 Map
obj_opt = phantom.build()

obj_opt.T1 = torch.ones_like(obj_opt.T1)    # reinitialize the current T1 map simply with 1

obj_opt.T1.requires_grad = True

meas_mask = torch.zeros_like(phantom.PD)
meas_mask[phantom.PD > 0] = 1
meas_mask = meas_mask[...,0]

optimizable_params = [
    {'params': obj_opt.T1, 'lr': 0.01}
]

target = torch.abs(space).clone()

with torch.no_grad():
    T1 = obj_opt.recover().T1

    plt.subplot(1,2,1)
    plt.imshow(T1,clim=(0,4))
    plt.title('Initial T1')
    plt.subplot(1,2,2)
    plt.imshow(T1_target,clim=(0,4))
    plt.title('T1 target')
    plt.show()


now we define our iteration step function, that generates the mr signal from the seq and the current T1, and compares this to the original target signal from above

In [ ]:
from jupyterplot import ProgressPlot
%matplotlib inline

loss_history = []
def calc_loss(seq0,iteration):

  signal = mr0.execute_graph(graph, seq0, obj_opt)
  kspace = torch.reshape((signal), (len(TI),Nphase, Nread)).clone().permute(0,2,1)
  kspace = kspace[:,:,ipermvec]
  spectrum = torch.fft.fftshift(kspace,dim=(1,2))
  space = torch.fft.fft2(spectrum)
  space = torch.fft.ifftshift(space,dim=(1,2))
  space_abs = torch.abs(space)
  reco_diff = (target*meas_mask[None] - space_abs*meas_mask[None])
  loss = torch.mean(reco_diff**2)
  loss_history.append(loss.detach().numpy())

  progress_plot.update(float(loss_history[-1])/float(loss_history[0]))

  with torch.no_grad():
    if iteration % 5 == 0:
      T1 = obj_opt.recover().T1
      update(T1,iteration)
  return loss



Now we optimize through the whole 5 TI IR FLASH

In [ ]:
# Create interactive plotting for loss curve and T1 maps
%matplotlib inline
fig = plt.figure()
fig.suptitle("Iteration: 0")
axes1 = fig.add_subplot(121)
axes2 = fig.add_subplot(122)

hfig = display(fig, display_id=True)

def draw():
  im1 = axes1.imshow(T1_target,clim=(0,4))
  c1 = plt.colorbar(im1)
  im1.set_clim(0,4)
  axes1.set_title('T1 target')
  im2 = axes2.imshow(T1,clim=(0,4))
  c2 = plt.colorbar(im2)
  im2.set_clim(0,4)
  axes2.set_title('Fit_pred')
  fig.canvas.draw()
  hfig.update(fig)

def update(T1,iteration):
  fig.suptitle("Iteration: " + str(iteration))
  axes1.cla()
  im1 = axes1.imshow(T1_target,clim=(0,4))
  axes1.set_title('T1 target')
  axes2.cla()
  im2 = axes2.imshow(T1,clim=(0,4))
  axes2.set_title('Fit_pred')
  fig.canvas.draw()
  hfig.update(fig)

draw()

loss_history = []

optimizer = torch.optim.Adam(optimizable_params)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=[0.5], total_steps=400)
iteration = 0

progress_plot = ProgressPlot(x_label = "Iteration", line_names = ["Loss"], width = 800, height = 500)

# Need a low iter count for building the documentation
for restart in range(10):
  torch.cuda.empty_cache()
  graph = mr0.compute_graph(seq0, obj_opt, 200, 1e-3)
  optimizer.zero_grad()
  loss = calc_loss(seq0, iteration)
  loss.backward()
  optimizer.step()
  scheduler.step()
  iteration += 1
progress_plot.finalize()
plt.close(fig)